# Setup

In [1]:
import pandas as pd
import os
import numpy as np
import json


In [2]:
data = pd.read_parquet(".." + os.sep + "data" + os.sep +
                "dati_presenze_politici.parquet")


trasformazione delle date in oggetti datetime

In [3]:
#data['DATA'] = data['DATA'].apply(lambda x: x.replace(" 00:00",""))
#data['DATA'] = data['DATA'].apply(lambda x: x.replace(":00",""))
#data.DATA = pd.to_datetime(data["DATA"],  format="%d.%m.%Y")


In [4]:
data.head(4)

,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia


In [5]:
def data2json(data, outfile, outpath=".." + os.sep + "docs" + os.sep + "data" + os.sep):
    json_data = json.dumps(data)
    json_file = open(outpath+outfile, "w")
    json_file.write(json_data)
    json_file.close()



# Soggetto collettivo

filtro per soggetti collettivo

In [16]:
dati_soggetti_colletivi = data[data.NOME =="Soggetto Collettivo"]
#dati_soggetti_colletivi = dati_soggetti_colletivi[dati_soggetti_colletivi.NOME ==
#                                        "Soggettivo Collettivo"]

## Distribuzione Interventi ed argomenti per Soggetti Collettivi

In [18]:
minuti_soggetti_collettivi = dati_soggetti_colletivi.groupby(["COGNOME"])["DURATA"].sum().reset_index().fillna(0).sort_values(by="DURATA", ascending=False)

In [19]:
categorie_argomento = list(dati_soggetti_colletivi.ARGOMENTO.unique())

In [9]:
color18 = [
    '#38b6b6', #1
    '#d5b158', #2
    '#3cb371', #3
    '#ff6666', #4
    '#6699ff', #5
    '#eaf889', #6
    '#0a915d', #7
    '#a092f1', #8 
    '#ff6347', #9
    '#82b6e9', #10
    '#2e4783', #11
    '#dcc392', #12
    '#d3758f', #13
    '#f3d999', #14
    '#a4d8c2', #15
    '#c6b38e',  #16
    '#b8d2c7', #17
    '#f5e8c8' #18
]

In [20]:
coloriCategorie = {}
for categoria in categorie_argomento:
    coloriCategorie[categoria] = color18[categorie_argomento.index(
        categoria)]


In [28]:
solo_politica = dati_soggetti_colletivi[dati_soggetti_colletivi.ARGOMENTO !=
                        "Politica e attività istituzionali"]
minuti_soggetti_collettivi_solo_politica = solo_politica.groupby(["COGNOME"])[
    "DURATA"].sum().reset_index().fillna(0).sort_values(by="DURATA", ascending=False)

In [29]:
minuti_soggetti_collettivi_solo_politica

,COGNOME,DURATA
40,Movimento 5 Stelle,44860
34,Lega Nord,34229
51,Partito Democratico,26280
48,PDL - Forza Italia,15086
17,Forza Nuova,7923
...,...,...
10,Centro Democratico-Italiani In Europa-Europeis...,2
54,Partito Socialista Italiano,2
4,Azione - +Europa ? Radicali Italiani,2
19,GAL Grandi autonomie e libertà,2


In [24]:
data_distro_interventi = {}
data_argomenti_movimenti = {}
oggetti_distro_interventi = []
label_notizia = "notizia"
label_intervento = "intervento"
for idx,row in minuti_soggetti_collettivi.iterrows():
    root = {}
    name = row.COGNOME
    root['name'] = name
    value = round(row.DURATA / 60,2)   
    root['value'] = value
    label = {}
    label['show'] = 'true'
    formatter = name + "\n" + str(value) + " ore\n"
    label['formatter'] = formatter
    root['label'] = label
    itemStyle = {}
    itemStyle['borderWidth'] =1
    itemStyle['borderColor'] = '#000000'
    itemStyle['opacity'] = 0.5
    root['itemStyle'] = itemStyle
    oggetti_distro_interventi.append(root)
    data_distro_interventi["dati"] = oggetti_distro_interventi
    
data2json(data_distro_interventi, "presenza_movimenti.json")

In [13]:
jump = "Politica e attivit\u00e0 istituzionali"
sankey_data_name = []
sankey_data_links = []
sankey_data = {}
for categoria in categorie_argomento:
    if categoria != jump:
        name_sankey = {}
        name_sankey['name'] = categoria
        sankey_data_name.append(name_sankey)
    addargomenti = True
    dati_soggetto_collettivo = data[data.COGNOME == name]
    argomenti = dati_soggetto_collettivo.groupby(["ARGOMENTO"])["DURATA"].sum(
    ).reset_index().fillna(0).sort_values(by="DURATA", ascending=False)
    for idx, row in argomenti.iterrows():
        oggetto = {}
        oggetto['source'] = name
        oggetto['target'] = row.ARGOMENTO
        oggetto['value'] = round(row.DURATA / 60, 2)
        itemstyle = {}
        itemstyle['color'] = coloriCategorie[row.ARGOMENTO]
        #oggetto['itemStyle'] = itemstyle
        sankey_data_links.append(oggetto)
       if row.ARGOMENTO != jump:
        name_sankey = {}
        name_sankey['name'] = name
        sankey_data_name.append(name_sankey)
sankey_data['data'] = sankey_data_name
sankey_data['links'] = sankey_data_links
    oggetti_distro_interventi.append(root)

data2json(sankey_data, "argomenti_movimenti.json")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 24)

# Personaggi politici

filtro per soli politici

In [ ]:
dati_politici = data[data.NOME != "Soggettivo Collettivo"]
dati_politici = dati_politici[dati_politici.NOME != "Soggetto Collettivo"]

creazione campo NOME_COGNOME

In [ ]:
dati_politici['NOME_COGNOME'] = dati_politici['NOME'] + \
    " " + dati_politici['COGNOME']


calcolo presenze per politico

In [ ]:
presenze = dati_politici.pivot_table(index="NOME_COGNOME",
                             values="DURATA", aggfunc=np.sum)#

In [ ]:
presenze.reset_index(inplace=True)


In [ ]:
presenze.sort_values(by="DURATA", ascending=False, inplace=True)

In [ ]:
totale_politici = len(dati_politici.NOME_COGNOME.unique())


In [ ]:
totale_politici


8547

ricerca più presenti nell'ultimo decile

In [ ]:
#ultimo decile
p = presenze.DURATA.quantile(0.9)

In [ ]:
presenze_politici_rilevanti = presenze[(presenze.DURATA > presenze.DURATA.quantile(0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


di questi prendiamo l'ultimo decile

In [ ]:
presenze_politici_top_rilevanti = presenze[(presenze.DURATA > presenze_politici_rilevanti.DURATA.quantile(
    0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


In [ ]:
presenze_politici_top_rilevanti.shape[0]

86

In [ ]:
presenze_politici_top_rilevanti

,NOME_COGNOME,DURATA
5887,Matteo Salvini,4830594
7830,Silvio Berlusconi,2542977
5001,Luigi Di Maio,2460683
5880,Matteo Renzi,1896863
3719,Giorgia Meloni,1835590
...,...,...
6929,Pietro Grasso,167791
5412,Maria Elena Boschi,165889
3641,Gianni Cuperlo,165194
4933,Luciano Nobili,160188


# Distribuzione argomenti

In [ ]:
distribuzione_argomenti = data.groupby(['DATA', 'ARGOMENTO','TIPO_TEMPO'])[
    'DURATA'].aggregate('sum').unstack().reset_index()

In [ ]:
distribuzione_argomenti.fillna(0, inplace=True)

In [ ]:
distribuzione_argomenti['TOTALE'] = distribuzione_argomenti['Notizia'] + distribuzione_argomenti['Parola']

In [ ]:
distribuzione_argomenti


TIPO_TEMPO,DATA,ARGOMENTO,Notizia,Parola,TOTALE
0,2018-01-01,Costume e società,16.0,0.0,16.0
1,2018-01-01,Cronaca,154.0,404.0,558.0
2,2018-01-01,"Economia, finanza e lavoro",11.0,0.0,11.0
3,2018-01-01,Giustizia,261.0,247.0,508.0
4,2018-01-01,Mass media e spettacolo,0.0,46.0,46.0
...,...,...,...,...,...
13197,2022-06-30,Costume e società,0.0,290.0,290.0
13198,2022-06-30,Cronaca,20.0,159.0,179.0
13199,2022-06-30,"Economia, finanza e lavoro",188.0,2706.0,2894.0
13200,2022-06-30,Esteri,109.0,1226.0,1335.0


In [ ]:
dati = []
radar_dati = {}


In [ ]:

totale_parole = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Parola'].aggregate('sum').to_frame().reset_index()
dati_interventi = {}
label_interventi = "Interventi"
interventi_argomenti_dati = []
for idx, row in totale_parole.iterrows():
    v = round(row.Parola / 60, 2)
    interventi_argomenti_dati.append(v)
dati_interventi['value'] = interventi_argomenti_dati
dati_interventi['name'] = label_interventi
dati.append(dati_interventi)


In [ ]:
totale_notizie = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Notizia'].aggregate('sum').to_frame().reset_index()

dati_notizie = {}
label_notizie = "Notizie"
notizie_argomenti_dati = []
out = "["
for idx, row in totale_notizie.iterrows():
    v = round(row.Notizia / 60,2)
    out += str(v) + ","
    notizie_argomenti_dati.append(v)
out = out[:-1] + "]"
dati_notizie['value'] = notizie_argomenti_dati
dati_notizie['name']=label_notizie
dati.append(dati_notizie)


In [ ]:
minuti_argomenti = distribuzione_argomenti.pivot_table(index="ARGOMENTO", values="TOTALE", aggfunc=np.sum).reset_index().sort_values(by="ARGOMENTO", ascending=True)

In [ ]:
indicators = []
for idx, row in minuti_argomenti.iterrows():
    blocco = {}
    name = row.ARGOMENTO
    value = round(row.TOTALE / 60,2)
    blocco['name'] = name 
    blocco['value'] = value
    out += "{ name: '%s', value: %s}," % (name, value)
    indicators.append(blocco)

In [ ]:
radar_dati = {}
radar_dati['titolo'] = "Distribuzione per argomenti"
radar_dati['titolo_radar'] = "Interventi vs Notizie"
radar_dati['indicatori'] = indicators
radar_dati['legenda'] = [label_interventi, label_notizie]
radar_dati['dati'] = dati


In [ ]:
data2json(radar_dati,"radar_argomenti.json")

# Flusso notizie/interventi nel tempo

In [ ]:
label_interventi = "INTERVENTI"
label_notizie = "NOTIZIE"
legenda = []
legenda.append(label_interventi)
legenda.append(label_notizie)
volume_notizie = {}
volume_notizie['titolo'] = "Ore di notizie ed interventi di politica nella televisione"
volume_notizie['legenda'] = legenda
dati = []


In [ ]:
volume_notizie_parole = data.groupby(['DATA', 'TIPO_TEMPO'])['DURATA'].aggregate('sum').unstack().reset_index()
volume_notizie_parole.sort_values(by="DATA", ascending=True, inplace=True)
volume_notizie_parole['TOTALE'] = volume_notizie_parole['Notizia'] + \
    volume_notizie_parole['Parola']


In [ ]:
volume_notizie_parole[volume_notizie_parole.TOTALE ==
                      volume_notizie_parole.TOTALE.max()]


TIPO_TEMPO,DATA,Notizia,Parola,TOTALE
1114,2021-01-19,18186,163020,181206


In [ ]:
for idx,row in volume_notizie_parole.iterrows():
    day = row.DATA.strftime('%Y/%m/%d')
    value_parole = []
    value_notizie = []
    parole = round(row.Parola / 60,2)
    notize = round(row.Notizia /60,2)
    value_parole.append(day)
    value_parole.append(parole)
    value_parole.append(label_interventi)
    value_notizie.append(day)
    value_notizie.append(notize)
    value_notizie.append(label_notizie)
    dati.append(value_parole)
    dati.append(value_notizie)


In [ ]:
volume_notizie['dati'] = dati

In [ ]:
data2json(volume_notizie, "volume_notizie.json")

In [ ]:
volume_notizie_parole[volume_notizie_parole.TOTALE ==
                      volume_notizie_parole.TOTALE.min()]


TIPO_TEMPO,DATA,Notizia,Parola,TOTALE
61,2018-03-03,463,883,1346


In [ ]:
data.COGNOME.unique()

array(['Berlusconi', 'Calenda', 'Conte', ..., 'Delli Colli', 'Calearo',
       'Gramazio'], dtype=object)